# Properties

This note book covers the basics of properties, how they are used, what inputs they accept,
and some examples.

## What is a property?

Properties are used by Features to determine how they are resolved. They represent a value,
which can be of any data type. This value can be updated through a sampling rule which is
passed to the class constructor on initialization. Concretely, a Property always has a value
accessible through the `current_value` field. This value is updated whenever `update()` is
called, and the new value is determined by the sampling rule passed as an input to the
constructor.

## What is a sampling rule?

A sampling rule, just like the `current_value` field, can be of any type. When a rule is
sampled - that is, when you request it to create a new value - it runs through a series of 
checks, executed in the folowing manner

1.  If the sampling rule has a callable `sample` method, call it and return the output.
2.  If the sampling rule is a dictionary, copy it and replace any value with a callable
    `sample` method with the output of that call.
3.  If the sampling rule is either a ``list`` or a 1-dimensional ``ndarray``, extract one
4.  If the sampling rule is an iterable, return the next value.
5.  If the sampling rule is callable, call it with no arguments and return the result.
    element
6.  If none of the above apply, return the sampling rule itself

Let's run som examples to showcase some of these cases, starting by importing the necessary classes



In [18]:
import sys
sys.path.append("..") # Adds the module to path
from DeepTrack.properties import Property

### 1) Constant value

The simplest example of a property is one that does not change during a update call.
This is commonly either a number or a tuple, but can be any data type that does not match any of the first five cases in the list. Note that any type can be made constant by wrapping it as the output of a lambda function.


In [19]:
P = Property(1)
print(P.current_value)
P.update()
print(P.current_value) # Numbers are constant over update() calls

1
1


### 1)  Discrete random variable

Discrete randomness can be achieved by either a list, a 1-dimensional ndarray or a function. For lists and ndarrays,
the output is uniformly sampled among the elements. For non-uniform sampling, either use lists with repeated elements, 
or a function. 

In [20]:
P = Property([1, 2, 3])
print([P.update().current_value for _ in range(10)])

TypeError: 'list' object is not an iterator